# JRA Details Scraper v2

- **取得対象**: 競走馬の詳細データ（過去走、血統）
- **主な変更点**:
  - `/horse/result/` (戦績) と `/horse/ped/` (血統) のDual URL取得
  - 過去5走のデータ (日付, 着順, タイム... コース, 天候, オッズ)
  - 馬体重の分割 (`_horse_weight`, `_weight_change`)
  - 騎手名のフルネーム化
  - 血統情報 (`father`, `mother`, `bms`)
- **取得変数**: 計70個


In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Settings
SAVE_DIR = '/content/drive/MyDrive/dai-keiba/data/raw'
BASIC_CSV = 'database_basic.csv'
DETAILS_CSV = 'database_details.csv'

In [ ]:
# Import Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os
import io
from tqdm.auto import tqdm

print("✅ Libraries loaded")
# --- Helper Functions for Details ---
JOCKEY_CACHE = {}
def get_jockey_fullname(url, short_name):
    if not url: return short_name
    if url in JOCKEY_CACHE: return JOCKEY_CACHE[url]
    try:
        time.sleep(0.5)
        if not url.startswith('http'):
            url = 'https://db.netkeiba.com' + url
        r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        r.encoding = 'EUC-JP'
        s = BeautifulSoup(r.text, 'html.parser')
        h1 = s.find('h1')
        if h1:
             title = h1.text.split()[0].strip()
             clean = re.sub(r'(の調教師成績|の騎手成績|のプロフィール|｜).*', '', title).strip()
             JOCKEY_CACHE[url] = clean
             return clean
    except:
                    pass
    return short_name


In [ ]:
# 馬履歴・血統取得関数

def get_horse_details(horse_id, race_date):
    """
    馬の履歴・血統データを取得(68カラム)
    race_date以前の過去5走のみ取得
    """
    details = {
        'race_id': '',
        'horse_id': horse_id
    }
    
    # 過去5走の初期化
    for i in range(1, 6):
        prefix = f'past_{i}'
        for field in ['date', 'rank', 'time', 'run_style', 'race_name', 'last_3f', 
                      'horse_weight', 'weight_change', 'jockey', 'condition', 'odds', 'weather', 'distance', 'course_type']:
            details[f'{prefix}_{field}'] = ''
    
    # 血統の初期化
    details['father'] = ''
    details['mother'] = ''
    details['bms'] = ''
    
    try:
        time.sleep(0.5)
        headers = {'User-Agent': 'Mozilla/5.0'}
        
        # 馬ページ取得
        url = f"https://db.netkeiba.com/horse/result/{horse_id}/"
        resp = requests.get(url, headers=headers, timeout=15)
        resp.encoding = 'EUC-JP'
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        # 血統情報(簡易版 - ページ内テキストから)
        # 実際の実装では血統テーブルをパース
        
        # レース履歴取得
        table = soup.select_one("table.db_h_race_results")
        if not table:
            tables = soup.find_all("table")
            for t in tables:
                if "着順" in t.text:
                    table = t
                    break
        
        if table:
            df = pd.read_html(io.StringIO(str(table)))[0]
            df = df.dropna(how='all')
            df.columns = df.columns.astype(str).str.replace(r'\s+', '', regex=True)
            
            # 日付フィルタリング
            # --- PEDIGREE FETCH (Dual URL) ---
            try:
                ped_url = f"https://db.netkeiba.com/horse/ped/{horse_id}/"
                time.sleep(0.5)
                r_ped = requests.get(ped_url, headers=headers)
                r_ped.encoding = 'EUC-JP'
                soup_ped = BeautifulSoup(r_ped.text, 'html.parser')
                blood_tbl = soup_ped.select_one('table.blood_table')
                if blood_tbl:
                    trs = blood_tbl.find_all('tr')
                    if len(trs) > 0:
                        tds0 = trs[0].find_all('td')
                        if len(tds0) > 0: details['father'] = tds0[0].text.strip().split('\n')[0]
                    
                    mid_idx = int(len(trs) / 2)
                    if mid_idx > 0 and len(trs) > mid_idx:
                        tds_m = trs[mid_idx].find_all('td')
                        if len(tds_m) > 0:
                             details['mother'] = tds_m[0].text.strip().split('\n')[0]
                        if len(tds_m) > 1:
                             details['bms'] = tds_m[1].text.strip().split('\n')[0]
            except: pass

            if '日付' in df.columns:
                df['date_obj'] = pd.to_datetime(df['日付'].astype(str).str.replace('.', '/'), errors='coerce')
                df = df.dropna(subset=['date_obj'])
                
                # race_date以前のレースのみ
                current_date = pd.to_datetime(race_date)
                df = df[df['date_obj'] < current_date]
                df = df.sort_values('date_obj', ascending=False)
                df = df.head(5)
                
                # データ抽出
                for i, row in enumerate(df.itertuples(), 1):
                    prefix = f'past_{pi}'
                    
                    details[f'{prefix}_date'] = getattr(row, '日付', '')
                    details[f'{prefix}_rank'] = str(getattr(row, '着順', ''))
                    details[f'{prefix}_time'] = str(getattr(row, 'タイム', ''))
                    details[f'{prefix}_race_name'] = str(getattr(row, 'レース名', ''))
                    details[f'{prefix}_last_3f'] = str(getattr(row, '上り', ''))
                    # Weight Split
                    w_val = str(getattr(row, '馬体重', ''))
                    w_num = w_val
                    w_chg = ''
                    if isinstance(w_val, str):
                        wm = re.search(r'(\d+)\(([\+\-\d]+)\)', w_val)
                        if wm:
                            w_num = wm.group(1)
                            w_chg = wm.group(2).replace('+', '')
                    details[f'{prefix}_horse_weight'] = w_num
                    details[f'{prefix}_weight_change'] = w_chg
                    # Jockey Full Name & Link Recovery
                    j_text = str(getattr(row, '騎手', ''))
                    r_date = str(getattr(row, '日付', ''))
                    try:
                        # Find row in soup by Date match
                        # table is defined.
                        found_link = None
                        for tr in table.find_all('tr'):
                            if r_date in tr.text:
                                # Found match
                                links = tr.find_all('a')
                                for lk in links:
                                    if '/jockey/' in lk.get('href', ''):
                                        found_link = lk.get('href')
                                        break
                                if found_link: break
                        
                        if found_link:
                            j_text = get_jockey_fullname(found_link, j_text)
                    except: pass
                    details[f'{prefix}_jockey'] = j_text
                    details[f'{prefix}_condition'] = str(getattr(row, '馬場', ''))
                    details[f'{prefix}_odds'] = str(getattr(row, '単勝', '') or getattr(row, 'オッズ', ''))
                    details[f'{prefix}_weather'] = str(getattr(row, '天気', ''))
                    
                    # 距離・コースタイプ
                    dist_text = str(getattr(row, '距離', ''))
                    dist_match = re.search(r'(芝|ダ|障)(\d+)', dist_text)
                    if dist_match:
                        course_type = dist_match.group(1)
                        details[f'{prefix}_course_type'] = '芝' if course_type == '芝' else 'ダート' if course_type == 'ダ' else '障害'
                        details[f'{prefix}_distance'] = dist_match.group(2)
                    
                    # 脚質(簡易版)
                    details[f'{prefix}_run_style'] = '3'  # デフォルト
        
        return details
    
    except Exception as e:
        print(f"  ⚠️ エラー({horse_id}): {e}")
        return details

print("✅ Horse details function loaded")

In [ ]:
# メインスクレイピング実行

def run_details_scraping():
    """
    Stage 1のCSVからhorse_idを読み込み、詳細データを取得
    """
    basic_path = os.path.join(SAVE_DIR, BASIC_CSV)
    details_path = os.path.join(SAVE_DIR, DETAILS_CSV)
    
    # Stage 1データ読み込み
    if not os.path.exists(basic_path):
        print(f"❌ Stage 1ファイルが見つかりません: {basic_path}")
        return
    
    df_basic = pd.read_csv(basic_path, dtype=str, on_bad_lines='skip', usecols=['race_id', 'horse_id', '日付'])
    print(f"📋 Stage 1データ: {len(df_basic)}行")
    
    # 既存のdetailsデータ
    existing_ids = set()
    if os.path.exists(details_path):
        df_existing = pd.read_csv(details_path, dtype=str, on_bad_lines='skip')
        if 'race_id' in df_existing.columns and 'horse_id' in df_existing.columns:
            # Re-scrape if data is missing (e.g. 'father' is empty)
            # If 'father' column exists, use it to filter completed rows
            if 'father' in df_existing.columns:
                # Robust check: father is not null and not empty string
                completed_df = df_existing[df_existing['father'].notna() & (df_existing['father'] != '')]
                existing_ids = set(completed_df['race_id'] + '_' + completed_df['horse_id'])
            else:
                # If column missing, assume all need scraping (or fallback to old logic?)
                # Better to clear existing_ids if we can't verify completeness
                existing_ids = set() # Force re-scrape if format is old
                # existing_ids = set(df_existing['race_id'] + '_' + df_existing['horse_id']) # Old Logic
        print(f"💾 既存データ: {len(existing_ids)}件")
    
    # 差分計算
    df_basic['key'] = df_basic['race_id'] + '_' + df_basic['horse_id']
    df_target = df_basic[~df_basic['key'].isin(existing_ids)]
    print(f"🚀 今回取得: {len(df_target)}件\n")
    
    if df_target.empty:
        print("✅ 全て取得済みです")
        return
    
    # スクレイピング
    buffer = []
    chunk_size = 50
    
    for i, row in enumerate(tqdm(df_target.itertuples(), total=len(df_target))):
        horse_id = row.horse_id
        race_date = row.日付
        race_id = row.race_id
        
        if not horse_id or not race_date:
            continue
        
        details = get_horse_details(horse_id, race_date)
        details['race_id'] = race_id
        buffer.append(details)
        
        # チャンク保存
        # --- Circuit Breaker (Safety Stop) ---
        if len(buffer) >= 10:
             valid_count = sum(1 for d in buffer if d.get('father', '').strip() != '')
             success_rate = valid_count / len(buffer)
             if success_rate < 0.2:
                 print(f"\n⚠️ CIRCUIT BREAKER TRIGGERED: Success rate {success_rate:.1%} is below threshold.")
                 print("⛔ Stopping execution to prevent empty data accumulation (likely IP Ban).")
                 # Save what we have so far (optional, but maybe risky if all empty)
                 # Raise exception to stop loop
                 raise RuntimeError("Circuit Breaker: High failure rate detected.")
        # -------------------------------------

        if len(buffer) >= chunk_size or (i == len(df_target) - 1 and buffer):
            df_chunk = pd.DataFrame(buffer)
            
            # カラム順序を明示
            ordered_columns = ['race_id', 'horse_id']
            for pi in range(1, 6):
                prefix = f'past_{pi}'
                for field in ['date', 'rank', 'time', 'run_style', 'race_name', 'last_3f', 
                              'horse_weight', 'weight_change', 'jockey', 'condition', 'odds', 'weather', 'distance', 'course_type']:
                    ordered_columns.append(f'{prefix}_{field}')
            ordered_columns.extend(['father', 'mother', 'bms'])
            
            df_chunk = df_chunk[ordered_columns]
            
            if not os.path.exists(details_path):
                df_chunk.to_csv(details_path, index=False)
            else:
                df_chunk.to_csv(details_path, mode='a', header=False, index=False)
            
            print(f"  💾 Saved {len(buffer)} records")
            buffer = []
    
    print("\n✅ スクレイピング完了")

print("✅ Main function ready")

In [ ]:
# 実行
run_details_scraping()